# 8-Weak Model vs Selected Model

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Create a Spark session
spark = SparkSession.builder.appName("LogisticRegressionModel").getOrCreate()

# Load data
csv_file_path = 'Merged-data.csv'
df = spark.read.format("csv").option("header", "true").load(csv_file_path)

# Using Binary target variable (1 or 0)
# If 'ClientsSeenRate' > threshold, set 'Target' to 1; otherwise, set it to 0.
threshold = 200
df = df.withColumn('Target', (df['ClientsSeenRate'] > threshold).cast("integer"))

# Select the features and target variable
selected_cols = ['Gender', 'AgeGroup', 'Ethnicity']
feature_cols = ['encoded_' + col for col in selected_cols]

# Encode categorical variables (Gender, AgeGroup, Ethnicity)
indexers = [StringIndexer(inputCol=col, outputCol='encoded_' + col).fit(df) for col in selected_cols]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

# Assemble features into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Split the data into training and testing sets
(train_data, test_data) = df.randomSplit([0.7, 0.3], seed=42)

# Create and train the logistic regression model
logistic_model = LogisticRegression(featuresCol="features", labelCol="Target")
model = logistic_model.fit(train_data)

# Make predictions on the test set
predictions = model.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(labelCol="Target", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)

# Print the results
print("Area under ROC = {}".format(accuracy))


# Confusion matrix
conf_matrix = predictions.groupBy("Target", "prediction").count()
conf_matrix.show()

# Extracting precision, recall, and F1 from confusion matrix
tp = conf_matrix.filter("Target = 1 AND prediction = 1").collect()[0]["count"]
fp = conf_matrix.filter("Target = 0 AND prediction = 1").collect()[0]["count"]
fn = conf_matrix.filter("Target = 1 AND prediction = 0").collect()[0]["count"]

precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)

print("Precision = {}".format(precision))
print("Recall = {}".format(recall))
print("F1 Score = {}".format(f1))

# Stop the Spark session
spark.stop()



Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/10 11:16:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/10 11:16:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Area under ROC = 0.689877250807834


23/10/10 11:16:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+----------+-----+
|Target|prediction|count|
+------+----------+-----+
|     1|       0.0|10818|
|     0|       0.0|45452|
|     1|       1.0|  575|
|     0|       1.0|  799|
+------+----------+-----+



Precision = 0.41848617176128095
Recall = 0.05046958658825595
F1 Score = 0.0900759771285345


In [2]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline

# Create a Spark session
spark = SparkSession.builder.appName("RandomForestClassifierModel").getOrCreate()

# Load data
csv_file_path = 'Merged-data.csv'
df = spark.read.format("csv").option("header", "true").load(csv_file_path)

# Using Binary target variable (1 or 0)
# If 'ClientsSeenRate' > threshold, set 'Target' to 1; otherwise, set it to 0.
threshold = 200
df = df.withColumn('Target', (df['ClientsSeenRate'] > threshold).cast("integer"))

# Select the features and target variable
selected_cols = ['Gender', 'AgeGroup', 'Ethnicity']
feature_cols = ['encoded_' + col for col in selected_cols]

# Encode categorical variables (Gender, AgeGroup, Ethnicity)
indexers = [StringIndexer(inputCol=col, outputCol='encoded_' + col, handleInvalid='keep', stringOrderType="frequencyDesc").fit(df) for col in selected_cols]
pipeline = Pipeline(stages=indexers)
df = pipeline.fit(df).transform(df)

# Assemble features into a vector
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df = assembler.transform(df)

# Split the data into training and testing sets
(train_data, test_data) = df.randomSplit([0.7, 0.3], seed=42)

# Create a Random Forest classifier with a seed
rf_classifier = RandomForestClassifier(featuresCol="features", labelCol="Target", seed=42)

# Create a parameter grid for cross-validation
param_grid = ParamGridBuilder() \
    .addGrid(rf_classifier.numTrees, [10, 20, 30]) \
    .addGrid(rf_classifier.maxDepth, [5, 10, 15]) \
    .build()

# Set up the cross-validator
evaluator = MulticlassClassificationEvaluator(labelCol="Target", predictionCol="prediction", metricName="accuracy")
cross_validator = CrossValidator(estimator=rf_classifier,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=4,  # We can adjust the number of folds as needed
                                 seed=42)

# Run cross-validation and choose the best set of parameters
cv_model = cross_validator.fit(train_data)

# Make predictions on the test set using the best model
cv_predictions = cv_model.transform(test_data)

# Fit the Random Forest model on the entire training set
final_model = rf_classifier.fit(train_data)

# Calculate accuracy
accuracy = evaluator.evaluate(cv_predictions)
print("Accuracy:", accuracy)


23/10/10 11:17:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Accuracy: 0.8138921657067518


In [3]:
confusion_matrix = cv_predictions.groupBy("Target", "prediction").count()
confusion_matrix.show()


+------+----------+-----+
|Target|prediction|count|
+------+----------+-----+
|     1|       0.0| 9995|
|     0|       0.0|45518|
|     1|       1.0| 1398|
|     0|       1.0|  733|
+------+----------+-----+

